# Introduction

Ce notebook est consacré à la réalisation d'un modèle de régression basé sur le LGBMRegressor (Light Gradient Boosting Machine), afin de prédire les ventes d'un ensemble de magasins. Pour cela, diverses données sont prises en compte : 

1. Les informations générales concernant les ventes et les magasins (fichier `train.csv`, `test.csv` et `stores.csv`), 
2. Les variations des prix du pétrole (`oil.csv`), facteur susceptible d'influencer le pouvoir d'achat des consommateurs,
3. Les jours fériés et événements spéciaux (`holidays_events.csv`) qui peuvent affecter les comportements d'achat,
4. Les transactions effectuées (`transactions.csv`).

Après avoir importé ces données, le notebook procède à une série de pré-traitements, incluant la fusion des différentes sources d'informations, le remplissage des valeurs manquantes, et la factorisation des données catégorielles. De plus, des caractéristiques temporelles supplémentaires sont ajoutées pour enrichir le modèle.

Une fois ces étapes de préparation effectuées, les données sont divisées en un ensemble d'entraînement et un ensemble de test, et un modèle LGBMRegressor est entraîné sur ces données. Les prédictions sont ensuite générées et évaluées à l'aide de l'erreur quadratique moyenne logarithmique. Enfin, les prédictions négatives sont ajustées à zéro, car une prédiction de vente ne peut être négative.

In [28]:
import pandas as pd
import numpy as np
import scipy as sp 
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

Ces lignes de code servent à importer plusieurs jeux de données à partir de fichiers CSV et à préparer ces données pour une analyse ultérieure.

1. `holidays = pd.read_csv('holidays_events.csv', parse_dates=True)`: Importe les données des événements spéciaux et des jours fériés, en s'assurant que toutes les dates sont correctement analysées comme des objets datetime.

2. `oil = pd.read_csv('oil.csv', parse_dates=True, index_col=0)`: Importe les données sur les prix de l'huile, tout en fixant la première colonne (qui est la date) comme l'index du DataFrame.

3. `sample_submission = pd.read_csv('sample_submission.csv')`: Importe le modèle de soumission qui est utilisé pour formater correctement nos prédictions avant de les soumettre pour évaluation.

4. `stores = pd.read_csv('stores.csv')`: Importe les informations détaillées sur chaque magasin.

5. `data_test = pd.read_csv('test.csv')` et `data_train = pd.read_csv('train.csv')`: Importent respectivement les données de test et d'entraînement. Ces données comprennent des informations détaillées sur chaque transaction.

6. `transactions = pd.read_csv('transactions.csv')`: Importe les informations sur toutes les transactions effectuées.

7. `data_test['date'] = pd.to_datetime(data_test.date)`, `data_train['date'] = pd.to_datetime(data_train.date)`, et `holidays['date'] = pd.to_datetime(holidays.date)`: Convertissent la colonne de date dans les DataFrames de test, d'entraînement et des jours fériés en datetime. Cela facilite le traitement ultérieur des dates et des heures.

In [2]:
holidays = pd.read_csv('holidays_events.csv', parse_dates=True)
oil = pd.read_csv('oil.csv', parse_dates=True, index_col=0)
sample_submission = pd.read_csv('sample_submission.csv')
stores = pd.read_csv('stores.csv')
data_test = pd.read_csv('test.csv')
data_train = pd.read_csv('train.csv')
transactions = pd.read_csv('transactions.csv')

In [3]:
data_test['date'] = pd.to_datetime(data_test.date)
data_train['date'] = pd.to_datetime(data_train.date)
holidays['date'] = pd.to_datetime(holidays.date)

Cette portion de code effectue plusieurs opérations de prétraitement sur les données :

1. **Création de l'index 'oil' :** Elle commence par la création d'un nouvel index pour le DataFrame 'oil'. Cet index est une combinaison des dates uniques présentes à la fois dans les DataFrames 'data_train' et 'data_test'. Les valeurs manquantes dans l'index 'oil' sont ensuite remplies en utilisant la méthode 'forward fill' et 'backward fill'.

2. **Fusion des DataFrames :** Les DataFrames 'data_train' et 'data_test' sont ensuite enrichis en fusionnant avec les DataFrames 'oil', 'stores', et 'holidays' sur les colonnes appropriées. La méthode 'left' est utilisée, ce qui signifie que toutes les lignes du DataFrame 'data_train' et 'data_test' sont conservées et seulement les lignes correspondantes des autres DataFrames sont utilisées.

3. **Traitement des valeurs manquantes :** Ensuite, toutes les colonnes présentant des valeurs manquantes dans 'data_test' sont identifiées. Ces valeurs manquantes sont remplies avec la valeur 'None' à la fois dans 'data_train' et 'data_test'.

4. **Factorisation des données :** Les DataFrames 'data_train' et 'data_test' sont ensuite concaténés en un seul DataFrame 'data_factorize', et toutes les colonnes de type 'object' sont factorisées. Cela signifie que les chaînes de caractères uniques sont remplacées par des nombres entiers, ce qui est souvent nécessaire pour les algorithmes de machine learning.

5. **Séparation des données d'entraînement et de test :** Le DataFrame 'data_factorize' est ensuite divisé à nouveau en 'data_train' et 'data_test' en utilisant l'index précédemment stocké ('test_idx').

6. **Ajout de caractéristiques temporelles :** Enfin, la fonction 'add_time_cols' est définie et utilisée pour ajouter des caractéristiques temporelles supplémentaires ('day', 'month', 'year') aux DataFrames 'data_train' et 'data_test'. La colonne 'date' est ensuite supprimée de ces DataFrames.

In [4]:
index_oil = pd.Index((pd.concat([pd.Series(data_train.date.unique()), pd.Series(data_test.date.unique())], axis=0)
                     .reset_index()
                     .drop(['index'], axis=1))[0])
index_oil.name = 'date'
oil = oil.reindex(index_oil).reset_index()
oil = oil.fillna(method='ffill').fillna(method='bfill')

In [5]:
data_train = pd.merge(data_train, oil.reset_index(), on='date', how='left').drop(['index'], axis=1)
data_test = pd.merge(data_test, oil.reset_index(), on='date', how='left').drop(['index'], axis=1)
data_train = pd.merge(data_train, stores, on='store_nbr', how='left')
data_test = pd.merge(data_test, stores, on='store_nbr', how='left')
data_train = pd.merge(data_train, holidays.drop_duplicates(subset='date'), on='date', how='left')
data_test = pd.merge(data_test, holidays.drop_duplicates(subset='date'), on='date', how='left')

In [6]:
data_train = data_train.set_index('id')
data_test = data_test.set_index('id')

In [7]:
na_col = [c for c in data_test.isna().any()[data_test.isna().any()].index]
data_train[na_col] = data_train[na_col].fillna(value='None')
data_test[na_col] = data_test[na_col].fillna(value='None')

In [8]:
test_idx = data_test.index[0]
data_factorize = pd.concat([data_train, data_test], axis=0)
for col, types in zip(data_factorize, data_factorize.dtypes):
    if types == 'object':
        data_factorize[col] = pd.factorize(data_factorize[col])[0]
        
data_test = data_factorize.iloc[test_idx:].drop(['sales'], axis=1)
data_train = data_factorize.iloc[:test_idx]
del data_factorize

In [20]:
def add_time_cols(data):
    data['day'] = data['date'].dt.dayofweek
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year
    data.drop(['date'], axis=1, inplace=True)
    return data

data_train, data_test = add_time_cols(data_train), add_time_cols(data_test)

Cette portion de code correspond à l'entraînement et à l'évaluation du modèle de régression :

1. **Séparation des caractéristiques et de la cible :** Les données d'entraînement sont divisées en caractéristiques (X) et cible (y). La cible est la colonne 'sales', que le modèle cherchera à prédire.

2. **Division des données en ensembles d'entraînement et de test :** Les données sont ensuite divisées en un ensemble d'entraînement et un ensemble de test en utilisant la fonction `train_test_split` de scikit-learn. 80% des données sont utilisées pour l'entraînement du modèle, et les 20% restants sont utilisés pour tester ses performances. L'option `shuffle=False` garantit que les données ne sont pas mélangées avant la division, ce qui est important pour conserver la structure temporelle des données.

3. **Entraînement du modèle :** Un modèle de régression LightGBM est ensuite entraîné sur l'ensemble d'entraînement. L'option `n_jobs=-1` permet d'utiliser tous les cœurs du processeur pour l'entraînement.

4. **Prédiction et évaluation :** Le modèle est utilisé pour prédire les ventes sur l'ensemble de test. Les prédictions négatives sont ajustées à 0, car une prédiction de vente ne peut être négative. Enfin, l'erreur quadratique moyenne logarithmique (mean squared log error) entre les valeurs prédites et les valeurs réelles est calculée et affichée.

5. **Vérification des prédictions :** Enfin, la valeur minimale des prédictions est calculée et affichée. Cela est probablement fait pour vérifier que les prédictions négatives ont bien été ajustées à 0.

In [25]:
X, y = data_train.drop(['sales'], axis=1), data_train['sales']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [52]:
model = LGBMRegressor(n_jobs=-1)
model.fit(X_train, y_train)
preds = model.predict(X_test)
preds = np.where(preds < 0, 0, preds)
print(mean_squared_log_error(y_test, preds))

3.118522077604036


In [49]:
np.where(preds < 0, 0, preds).min()

125.5684359719774